In [ ]:
from mg_utils import *

# Drift Equilibrium
Fixed: β, r

Eq: V, Fx, δ

In [ ]:
# parameters and initial conditions
# FORCE_REAR = 30 # [N] constant rear longitudinal force per wheel
# STEER = 0.35 # [rad] constant steering angle
# DELTA, FX = STEER * steering_ratio_f, 2 * FORCE_REAR # wheel angle and longitudinal force

DELTA, FX = -15*π/180, 20 # wheel angle and longitudinal force
V0, BETA0, R0 = 4, -25*π/180, 1.2 # [m/s], [rad], [rad/s] initial velocity, sideslip angle, and yaw rate
# V0, BETA0, R0 = 4, -25*π/180, 0.5 # [m/s], [rad], [rad/s] initial velocity, sideslip angle, and yaw rate
# V0, BETA0, R0 = 4, 5*π/180, 1 # [m/s], [rad], [rad/s] initial velocity, sideslip angle, and yaw rate

T = 4 # [s] total simulation time
DT = 0.001 # [s] time step for the simulation
N_STEPS = int(T / DT) # number of steps in the simulation


print(f'Max Force: {μr * Fz_Rear :.2f} [N]')

In [ ]:
# equation of motion for the Single Track Model
def d_vβr(vβr, δ, Fx): 
    v, β, r = vβr # unpack the state vector
    # assert v >= 0, "Velocity must be non-negative" # ensure velocity is non-negative
    if v < 0.001: v = 0.001 # avoid division by zero
    Fyf = tire(f_αf(δ,v,β,r), 0.0, Fz_Front, μf, Cyf) # lateral force front
    Fyr = tire(f_αr(δ,v,β,r), Fx, Fz_Rear, μr, Cyr) # lateral force rear
    Fxr = Fx # rear longitudinal force
    return np.array([ # equations of motion
        (-Fyf*np.sin(δ-β) + Fxr*np.cos(β) + Fyr*np.sin(β)) / m, # V dot
        (+Fyf*np.cos(δ-β) - Fxr*np.sin(β) + Fyr*np.cos(β)) / (m*v) - r, # β dot
        (a*Fyf*np.cos(δ) - b*Fyr) / J_CoG # r dot
    ])

def stm_rk4(vβr, Fx, δ, dt=DT): # runge-kutta 4th order method
    k1 = d_vβr(vβr, δ, Fx) * dt
    k2 = d_vβr(vβr + k1/2, δ, Fx) * dt
    k3 = d_vβr(vβr + k2/2, δ, Fx) * dt
    k4 = d_vβr(vβr + k3, δ, Fx) * dt
    return vβr + (k1 + 2*k2 + 2*k3 + k4) / 6 # update the state vector

# initialize the state vector
state = np.zeros((N_STEPS, 3)) # [u, v, r]
state[0] = np.array([V0, BETA0, R0]) # initial state in v,β,r format
print(f"Initial state: {state[0]} [v,β,r]") # print the initial state in v,β,r format

# initialize the u input vector
u = np.zeros((N_STEPS, 2)) # [Fx, δ]
u[:,0], u[:,1] = FX, DELTA # assign constant values
# run the simulation
for i in range(1, N_STEPS):
    state[i] = stm_rk4(state[i-1], u[i, 0], u[i, 1])  
print(f"Final state:   {state[-1]} [v,β,r]") # print the final state in v,β,r format

# plot first n steps of the simulation  
dec = 10 # decimation factor
v1, β1, r1 = state[:,0], state[:,1], state[:,2] # unpack the state vector
v1, β1, r1 = v1[::dec], β1[::dec], r1[::dec] # decimate the state vector
plt.figure(figsize=(8, 3))
plt.plot(v1, label='v1 (velocity)')
plt.plot(β1, label='β1 (side slip angle)')
plt.plot(r1, label='r1 (yaw rate)')
plt.title('STM Simulation Results')
plt.xlabel('Time Step')
plt.ylabel('State Variables')
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
# # car animation
# anim = car_anim(
#     vβrs=state,  # use the state vector as input
#     δs=u[:,1],  # use the u input vector for steering angles
#     ic=np.array([0, 0,π/2]),  # initial conditions (x, y, ψ) 
#     dt=DT,  # time step
#     fps=60,  # frames per second
#     speed=1.0  # speed factor for the animation
# )  # run the car animation with the STM results

In [ ]:
# same as before, but using casadi for symbolic differentiation
import casadi as ca
from casadi import SX, sqrt, atan, sin, cos, tanh, atan2, vertcat
from acados_template import AcadosModel, AcadosOcp, AcadosOcpSolver, AcadosSim, AcadosSimSolver

# variables
v = SX.sym('v') # velocity
v̇ = SX.sym('v̇') # velocity dot
β = SX.sym('β') # sideslip angle
β̇ = SX.sym('β̇') # sideslip angle dot
r = SX.sym('r') # yaw rate
ṙ = SX.sym('ṙ') # yaw rate dot
δ = SX.sym('δ') # wheel angle (on the road)
Fx = SX.sym('Fx') # rear longitudinal force

x = vertcat(v, β, r) # state vector
ẋ = vertcat(v̇, β̇, ṙ) # state dot vector
u = vertcat(Fx, δ) # u input vector 

# tire model
αf = δ - atan2(v*sin(β) + a*r, v*cos(β))
αr = -atan2(v*sin(β) - b*r, v*cos(β))
def fiala_tanh_ca(α, Fx, Fz, μ, Cy):
    Fy_max = sqrt(μ**2 * Fz**2 - Fx**2) # maximum lateral force
    αs = atan(Fy_max/Cy) # maximum slip angle
    return Fy_max * tanh(α / αs) # tanh approximation

# lateral forces
Fyf = fiala_tanh_ca(αf, 0.0, Fz_Front, μf, Cyf) # lateral force front
Fyr = fiala_tanh_ca(αr, Fx, Fz_Rear, μr, Cyr) # lateral force rear
Fxr = Fx # rear longitudinal force

# define the symbolic equations of motion
dv = (-Fyf*sin(δ-β) + Fxr*cos(β) + Fyr*sin(β)) / m # V dot
dβ = (+Fyf*cos(δ-β) - Fxr*sin(β) + Fyr*cos(β)) / (m*v) - r # β dot
dδ = (a*Fyf*cos(δ) - b*Fyr) / J_CoG # r dot

dx = vertcat(dv, dβ, dδ) # equations of motion

# create the model
model = AcadosModel()
model.name='stm_model'
model.f_impl_expr = ẋ - dx  
model.f_expl_expr = dx 
model.x = x  # state vector
model.xdot = ẋ  # state dot vector
model.u = u  # u input vector

In [ ]:
# test Acados Integrator
sim1step = AcadosSim()
sim1step.model = model  # set the model
sim1step.solver_options.T = DT  # 
sim1step.solver_options.integrator_type = 'ERK'
acados_integrator = AcadosSimSolver(sim1step)  # create the integrator

# test the integrator
xs = np.zeros((N_STEPS, 3)) # [v, β, r]
xs[0] = np.array([V0, BETA0, R0]) # initial state in v,β,r format
us = np.zeros((N_STEPS, 2)) # [Fx, δ]
us[:,0], us[:,1] = FX, DELTA # assign constant values
for i in range(1, N_STEPS):
    xs[i] = acados_integrator.simulate(x=xs[i-1], u=us[i-1])  # simulate one step

# # car animation
# anim = car_anim(
#     vβrs=xs,  # use the state vector as input
#     δs=us[:,1],  # use the u input vector for steering angles
#     ic=np.array([0, 0,π/2]),  # initial conditions (x, y, ψ) 
#     dt=DT,  # time step
#     fps=60,  # frames per second
#     speed=1.0  # speed factor for the animation
# )  # run the car animation with the STM results

print(f"Final state:   {xs[-1]} [v,β,r]") # print the final state in v,β,r format

## Finding equilibrium point with Casadi
Fixed: β, r

Optim: V, Fx, δ

In [ ]:
# constraints
MAX_DELTA = 20 * π / 180  # [rad] maximum steering angle in radians
MAX_V = 10 # [m/s] maximum velocity
# MAX_FX = 0.8 * μr*Fz_Rear # [N] maximum rear longitudinal force
MAX_FX = 30 # [N] maximum rear longitudinal force

print(f'Constraints: MAX_DELTA={MAX_DELTA:.2f}, MAX_V={MAX_V:.2f}, MAX_FX={MAX_FX:.2f}')

In [ ]:
# Define optimization problem to find steady-state (dx == 0)
# test finding zeros of the equations of motion, using casadi
print(f'eqs: {dx}')

# substitute fixed variables with their values
f = dx 
f = ca.substitute(f, β, BETA0)  # substitute β
f = ca.substitute(f, r, R0)  # substitute r
opt_vars = vertcat(v, Fx, δ)  # optimization variables
opt_func = ca.Function('opt_func', [opt_vars], [f])  # create a function for the equations of motion
#constraints # abs(delta) < 0.35,  0 < Fx < 50, abs(v) < 20
g = vertcat(δ,Fx,v)
ubg = vertcat(MAX_DELTA, MAX_FX, MAX_V)  # upper bounds for the constraints
lbg = vertcat(-MAX_DELTA, 0, -MAX_V)  #

nlp = { # NLP -> minimize sum of squares of residuals
    'x': opt_vars, # optimization variables
    'f': ca.sumsqr(opt_func(opt_vars)), # objective function (sum of squares of residuals)
    'g': g
}  
solver = ca.nlpsol('solver', 'ipopt', nlp) # solver for the NLP
initial_guess = [V0, FX, DELTA]  # initial guess for the optimization variables
sol = solver(x0=initial_guess, lbg=lbg, ubg=ubg)  # solve the NLP with constraints

print(f'Solution -> v={sol["x"][0]}, Fx={sol["x"][1]}, δ={sol["x"][2]}')  # Print solution
print(f'Function value (residuals): {sol["f"]}')  # Print function value

# simulate the STM with the solution
# create the vector of initial conditions
eq_vβr = [float(sol["x"][0]), float(BETA0), R0]  # use the solution for v, keep β and r fixed
eq_Fx, eq_δ = float(sol["x"][1]), float(sol["x"][2])  # use the solution for Fx and δ

# print the equilibrium point
print(f'Equilibrium point: v={eq_vβr[0]}, β={eq_vβr[1]}, r={eq_vβr[2]}, Fx={eq_Fx}, δ={eq_δ}')  # print the equilibrium point

In [ ]:
# simulate the STM with the solution
#let's do a perfect circle
r_turn = eq_vβr[0]/eq_vβr[2]  # radius of the turn
print(f'Radius of the turn: {r_turn:.2f} [m]')
full_circle = 2*π*r_turn  # full circle length
sim_time = full_circle / eq_vβr[0]  # time to complete a full circle
print(f'Time to complete a full circle: {sim_time:.2f} [s]')


xs = sim_stm_fixed_u(
    vβr0=eq_vβr,  # use the equilibrium point as initial state
    Fx=eq_Fx,  # use the solution for Fx
    δ=eq_δ,  # use the solution for δ
    sim_t=sim_time,  # simulate for 2 seconds
    dt=DT,  # time step
)  # run the simulation with the equilibrium point
 
# animation
anim = car_anim(
    vβrs=xs,  # use the state vector as input 
    δs=np.full(xs.shape[0], eq_δ),  # use the solution for δ
    ic=np.array([0, 0, np.pi/2]),  # initial conditions (x, y, ψ) 
    dt=DT,  # time step
    fps=60,  # frames per second
    speed=0.75,  # speed factor for the animation
    # follow=True,  # follow the car in the animation
)  # run the car animation with the STM results